In [13]:
%%time
# Program makes the pre-order dataframe

# STATUS: Complete
# Runtime: 5 secs

#***          Start ib_insync (run once)       *****
#___________________________________________________

from ib_insync import *
util.startLoop()
ib=IB().connect('127.0.0.1', 7496, clientId=7) # kavi TWS live
# ib=IB().connect('127.0.0.1', 4001, clientId=7) # kavi IBG live

import pandas as pd
import numpy as np

#******         Paths and variables         ****
#_______________________________________________

datapath = r'./zdata/'
th_file = r'./put_threshold1.csv'  # thresold file with min close values
po_file = r'./df_preorder_puts.pkl' # 

# Read the put thresholds
df_th = pd.read_csv(datapath+th_file, sep=',', 
            header=None).rename(columns={1: 'ibSymbol', 4: 'Threshold'}).drop([0,2,3],1)

df_th.Threshold = df_th.Threshold.astype(float)

# Read the preorder puts
df_puts = pd.read_pickle(datapath+'df_preorder_puts.pkl')

# Filter based on thresholds
cols = ['ibSymbol', 'undPrice', 'Strike', 'Expiry', 'DTE', 
        'pLTP', 'pBidPrice', 'tgt_price', 'Type', 'Mlot', 'Delta', 'RoM']
df_puts1 = df_puts[cols]
df_puts2 = df_puts1.merge(df_th)
df_puts3 = df_puts2[df_puts2.Strike <= df_puts2.Threshold].reset_index(drop=True)

# convert expiry to the format required by IBKR
df_puts3.Expiry = df_puts3.Expiry.dt.strftime('%Y%m%d')

#****       Placing the Orders    ********

# Qualify the contracts and put them into a dataframe
c_list = [Option(symbol=symbol, exchange='NSE', 
       lastTradeDateOrContractMonth=expiry, strike=str(strike), right='P') 
for symbol, strike, expiry in zip(df_puts3.ibSymbol, df_puts3.Strike, df_puts3.Expiry)]

contracts = [c for i in range(0, len(c_list), 50) 
             for c in ib.qualifyContracts(*c_list[i: i+50])]

cols = ['ibSymbol', 'Expiry', 'Strike', 'Type']
df_contracts = pd.DataFrame([(c.symbol, c.lastTradeDateOrContractMonth, c.strike, c.right, c) 
              for c in contracts], columns=cols+['Contract'])

# prepare to make the orders
df_orders = pd.merge(df_puts3, df_contracts, on=cols).sort_values(['RoM'], 
                    ascending=False).reset_index(drop=True)

df_orders['Orders'] = [LimitOrder(action='SELL', totalQuantity=mlot, lmtPrice=price) 
for mlot, price in zip(df_orders.Mlot, df_orders.tgt_price)]

df_orders.to_pickle(datapath+'df_orders.pkl')

ib.disconnect()

Started to throttle requests
Stopped to throttle requests
Error 200, reqId 294: No security definition has been found for the request, contract: Option(symbol='INDIACEM', lastTradeDateOrContractMonth='20181227', strike='75.0', right='P', exchange='NSE')
Error 200, reqId 295: No security definition has been found for the request, contract: Option(symbol='INDIACEM', lastTradeDateOrContractMonth='20181227', strike='70.0', right='P', exchange='NSE')
Unknown contract: Option(symbol='INDIACEM', lastTradeDateOrContractMonth='20181227', strike='75.0', right='P', exchange='NSE')
Unknown contract: Option(symbol='INDIACEM', lastTradeDateOrContractMonth='20181227', strike='70.0', right='P', exchange='NSE')
Error 200, reqId 359: No security definition has been found for the request, contract: Option(symbol='KTKBANK', lastTradeDateOrContractMonth='20181227', strike='90.0', right='P', exchange='NSE')
Error 200, reqId 360: No security definition has been found for the request, contract: Option(symbol=

Wall time: 16 s
